In [1]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import nibabel as nib
from scipy import spatial
from nibabel import freesurfer


In [2]:
def ensure_dir(file_name):
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    return file_name


def time_str(mode='abs', base=None):
    if mode == 'rel':
        return str(datetime.timedelta(seconds=(time.time() - base)))
    if mode == 'raw':
        return time.time()
    if mode == 'abs':
        return time.asctime(time.localtime(time.time()))


In [77]:
cortical_atlas_names = [
    'Glasser', 'Schaefer7n100p', 'Schaefer7n200p', 'Schaefer7n300p', 'Schaefer7n400p', 'Schaefer7n500p',
    'Schaefer7n600p', 'Schaefer7n700p', 'Schaefer7n800p', 'Schaefer7n900p', 'Schaefer7n1000p', 'Schaefer17n100p',
    'Schaefer17n200p', 'Schaefer17n300p', 'Schaefer17n400p', 'Schaefer17n500p', 'Schaefer17n600p', 'Schaefer17n700p',
    'Schaefer17n800p', 'Schaefer17n900p', 'Schaefer17n1000p', 'aparc', 'aparc.a2009s',
]
subcortical_atlas_names = [
    "Tian_Subcortex_S1_3T", "Tian_Subcortex_S2_3T", "Tian_Subcortex_S3_3T", "Tian_Subcortex_S4_3T",
]
cols = ['index', 'label_name']
for atlas_1_name in cortical_atlas_names:
    for atlas_2_name in subcortical_atlas_names:
        cortical_labels = pd.DataFrame(
            np.genfromtxt(
                '{}/atlases/labels/{}.ColorLUT.txt'.format(template_dir, atlas_1_name),
                dtype='str'
            ),
            columns=['index', 'label_name', 'R', 'G', 'B', 'A'],
        ).astype(
            dtype={
                "index": "int",
                "label_name": "str",
                "R": "int",
                "G": "int",
                "B": "int",
                "A": "int",
            }
        )
        
        updated_cortical_labels = cortical_labels.copy()
        updated_cortical_labels['label_name'] = [(f'{atlas_1_name}_' + x if x!='???' else x) for x in cortical_labels['label_name']]

        subcortical_labels = pd.DataFrame(
            ['???'] + list(np.genfromtxt(
                '{}/atlases/labels/{}_label.txt'.format(template_dir, atlas_2_name),
                dtype='str'
            )),
            columns=['label_name'],
        ).astype(
            dtype={
                "label_name": "str",
            }
        )
        subcortical_labels['index'] = subcortical_labels.index
        
        updated_subcortical_labels = subcortical_labels.copy()
        updated_subcortical_labels['index'] = [((x + cortical_labels['index'].max()) if x!=0 else 0) for x in subcortical_labels['index']]
        updated_subcortical_labels['label_name'] = f'{atlas_2_name}_' + subcortical_labels['label_name']
        
        combined_labels = pd.concat([updated_cortical_labels[cols], updated_subcortical_labels[1:][cols]])
        
        np.savetxt(
            ensure_dir('{}/atlases/labels/combinations/C:{}+S:{}_label.txt'.format(template_dir, atlas_1_name, atlas_2_name)),
            np.array(combined_labels[cols]),
            fmt=['%d', '%s',]
        )


In [3]:
atlas_1_name = 'aparc'
atlas_2_name = 'Tian_Subcortex_S4_3T'


In [6]:
cortical_labels = pd.DataFrame(
    np.genfromtxt(
        '{}/atlases/labels/{}.ColorLUT.txt'.format(template_dir, atlas_1_name),
        dtype='str'
    ),
    columns=['index', 'label_name', 'R', 'G', 'B', 'A'],
).astype(
    dtype={
        "index": "int",
        "label_name": "str",
        "R": "int",
        "G": "int",
        "B": "int",
        "A": "int",
    }
)


In [8]:
template_dir = '/home/sina/Documents/Research/Codes/UKB-connectomics/data/templates'
ukb_subjects_dir = '/home/sina/Documents/Research/Datasets/UK_biobank/sample'
ukb_subject_id = '1000243'
ukb_instance = '2_0'

atlas_1 = f'{ukb_subjects_dir}/{ukb_subject_id}_{ukb_instance}/dMRI/dMRI/atlases/native.dMRI_space.{atlas_1_name}.nii.gz'
atlas_2 = f'{ukb_subjects_dir}/{ukb_subject_id}_{ukb_instance}/dMRI/dMRI/atlases/native.dMRI_space.{atlas_2_name}.nii.gz'
atlas_1_image = nib.as_closest_canonical(nib.load(atlas_1))
atlas_2_image = nib.as_closest_canonical(nib.load(atlas_2))

# load atlas data
atlas_1_data = atlas_1_image.get_fdata()
atlas_2_data = atlas_2_image.get_fdata()

# mask and shift atlas_2 labels
data_mask = (atlas_1_data == 0)
shift_mask = (atlas_2_data > 0)
shift_value = cortical_labels['index'].max()
atlas_2_data_masked_shifted = np.multiply((shift_mask & data_mask), (atlas_2_data + shift_value))

combined_atlas_data = atlas_1_data + atlas_2_data_masked_shifted


In [132]:
(shift_mask & ~data_mask).sum()

3054

In [9]:
# atlas_1_image.affine, atlas_2_image.affine
# atlas_2_image.header.values()
nib.aff2axcodes(atlas_1_image.affine), nib.aff2axcodes(atlas_2_image.affine)


(('R', 'A', 'S'), ('R', 'A', 'S'))

In [13]:
new_header = atlas_1_image.header.copy()
new_header.set_slope_inter(1, 0)

In [14]:
new_header.get_slope_inter()

(1.0, 0.0)

In [18]:
new_image = nib.Nifti1Image(combined_atlas_data, atlas_1_image.affine, new_header)

In [20]:
new_header.get_slope_inter()

(1.0, 0.0)

In [23]:
dict(new_header)

{'sizeof_hdr': array(348, dtype=int32),
 'data_type': array(b'', dtype='|S10'),
 'db_name': array(b'transform modifie', dtype='|S18'),
 'extents': array(16384, dtype=int32),
 'session_error': array(0, dtype=int16),
 'regular': array(b'r', dtype='|S1'),
 'dim_info': array(0, dtype=uint8),
 'dim': array([  3, 256, 256, 256,   1,   1,   1,   1], dtype=int16),
 'intent_p1': array(0., dtype=float32),
 'intent_p2': array(0., dtype=float32),
 'intent_p3': array(0., dtype=float32),
 'intent_code': array(0, dtype=int16),
 'datatype': array(768, dtype=int16),
 'bitpix': array(32, dtype=int16),
 'slice_start': array(0, dtype=int16),
 'pixdim': array([1.        , 1.        , 1.        , 0.99999994, 0.        ,
        0.        , 0.        , 0.        ], dtype=float32),
 'vox_offset': array(0., dtype=float32),
 'scl_slope': array(1., dtype=float32),
 'scl_inter': array(0., dtype=float32),
 'slice_end': array(0, dtype=int16),
 'slice_code': array(0, dtype=uint8),
 'xyzt_units': array(10, dtype=uint

In [19]:
new_image.header.get_slope_inter()

(None, None)

In [26]:
new_image.header.set_slope_inter(1, 0)

In [27]:
dict(new_image.header)

{'sizeof_hdr': array(348, dtype=int32),
 'data_type': array(b'', dtype='|S10'),
 'db_name': array(b'transform modifie', dtype='|S18'),
 'extents': array(16384, dtype=int32),
 'session_error': array(0, dtype=int16),
 'regular': array(b'r', dtype='|S1'),
 'dim_info': array(0, dtype=uint8),
 'dim': array([  3, 256, 256, 256,   1,   1,   1,   1], dtype=int16),
 'intent_p1': array(0., dtype=float32),
 'intent_p2': array(0., dtype=float32),
 'intent_p3': array(0., dtype=float32),
 'intent_code': array(0, dtype=int16),
 'datatype': array(768, dtype=int16),
 'bitpix': array(32, dtype=int16),
 'slice_start': array(0, dtype=int16),
 'pixdim': array([1.        , 1.        , 1.        , 0.99999994, 0.        ,
        0.        , 0.        , 0.        ], dtype=float32),
 'vox_offset': array(0., dtype=float32),
 'scl_slope': array(1., dtype=float32),
 'scl_inter': array(0., dtype=float32),
 'slice_end': array(0, dtype=int16),
 'slice_code': array(0, dtype=uint8),
 'xyzt_units': array(10, dtype=uint

In [136]:
nib.save(
    nib.Nifti1Image(combined_atlas_data, atlas_1_image.affine, atlas_1_image.header),
    f'{ukb_subjects_dir}/{ukb_subject_id}_{ukb_instance}/dMRI/dMRI/native.dMRI_space.{atlas_1_name}+{atlas_2_name}.nii.gz'
)